In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy.stats import norm

from funcs import ssa, gauss_reaction_matrix, initial_pop_gaussian

# Simulation 

In [ ]:
############   Parameters 

device = 'cpu'

n_experiments = 1000
n_iterations_list = np.array([100,200,400])                 # number of iterations; each iteration is (vaccination+wait) or (growth followed by mutation/death)

L = 21
n_bins = 2*L+1  # number of affinity bins; x bins are {-L,-L+1,...,0,...,L-1,L} with indexes {0,1,...,L,...,2L}
rho = 1e-3   # death rate
mu = 0.1     # total (left+right) mutation right
D = mu/2     # the equiv diffusion const is D = (1/2) \int y^2 \mu(y) ---> 2.5e-2
q = 0.5      # outward mutation rates are q*mu

F_tot_list = np.array([0.04, 0.08])                          # various total fitnesses to be tested separately
pop0_center_list = L + np.array([4, 6, 8, 10, 12, 14])       # where the initial population is centered at; these are the indexes in an array that is indexed 0 for -L, hence "L+"
pop0_sigma_list = np.array([1,3])                            # width of the initial population (Gaussian shape)
pop0_tot = 10                                                # initial total population
max_population = 2000   

max_time =  400      # based on the condition that |mu* - mu0| < sigma0; 2D = mu
max_time_simulation = 2 * max_time
# factor of 2: each iteration in the simulation has two regimes - whereas in reality both growth and mutation happen during the same interval (trotter only seperates their operators)
# this will affect trajectories - because would be saved at the end of each 'regime'; but the final n is not affected

In [ ]:
for Ftot_index in np.arange(F_tot_list.size):
    for pop0_center_index in np.arange(pop0_center_list.size):
        for pop0_sigma_index in np.arange (pop0_sigma_list.size): 
            
            F_tot = F_tot_list[Ftot_index]
            pop0_center = -L + pop0_center_list[pop0_center_index]    
            pop0_sigma = pop0_sigma_list[pop0_sigma_index]
            pop0 = initial_pop_gaussian(n_bins, pop0_tot, L+pop0_center, pop0_sigma)
            
            for n_iterations_index in np.arange (n_iterations_list.size):
                
                n_iterations = n_iterations_list[n_iterations_index]
                n_regimes = 2 * n_iterations
                print(f'Ftot={F_tot}, mu0={pop0_center}, sigma0={pop0_sigma}, rounds={n_iterations} ')
                
                dt_fit = max_time/n_iterations
                dt_mu = max_time/n_iterations
                
                F_per_round = np.ones(n_iterations)*(F_tot)
                
                rxn_tEnd = np.cumsum(np.tile([dt_fit, dt_mu], n_iterations))   

                #######  fitness width and center
                radii_cnst =  np.ones(n_iterations)
                radii_lin =  (3*D*dt_mu) * np.arange(n_iterations, 0, -1)      # vaccine width shrinking linearly
                radii_BCH = np.maximum ( np.sqrt(radii_lin), radii_cnst)       # vaccine width shrinking diffusively
                
                centers_xstar = L + (pop0_center/(1+ ( pop0_sigma**2/(n_iterations*2*D*dt_mu) ))) * (1- (np.arange(1,n_iterations+1)/n_iterations))
                centers_x12   = L + (pop0_center/(1+ ( pop0_sigma**2/(n_iterations*2*D*dt_mu) ))) * (1- (np.arange(1,n_iterations+1)/n_iterations))**2
                centers_x21   = L + (pop0_center/(1+ ( pop0_sigma**2/(n_iterations*2*D*dt_mu) ))) * (1- (np.arange(1,n_iterations+1)/n_iterations)**2)  
                centers_x105  = L + (pop0_center/(1+ ( pop0_sigma**2/(n_iterations*2*D*dt_mu) ))) * (1- (np.arange(1,n_iterations+1)/n_iterations))**(0.5)
                centers_x051  = L + (pop0_center/(1+ ( pop0_sigma**2/(n_iterations*2*D*dt_mu) ))) * (1- (np.arange(1,n_iterations+1)/n_iterations)**(0.5))
                
                
                #############   create rxn matrices 
                
                gauss_xstar_r1 = gauss_reaction_matrix (n_bins, n_iterations, centers_xstar,radii_cnst, F_per_round, rho, mu, q)
                gauss_xstar_rBCH = gauss_reaction_matrix (n_bins, n_iterations, centers_xstar, radii_BCH, F_per_round, rho, mu, q)
                gauss_x12_r1 = gauss_reaction_matrix (n_bins, n_iterations, centers_x12,radii_cnst, F_per_round, rho, mu, q)
                gauss_x21_r1 = gauss_reaction_matrix (n_bins, n_iterations, centers_x21,radii_cnst, F_per_round, rho, mu, q)
                gauss_x105_r1 = gauss_reaction_matrix (n_bins, n_iterations, centers_x105 ,radii_cnst, F_per_round, rho, mu, q)
                gauss_x051_r1 = gauss_reaction_matrix (n_bins, n_iterations, centers_x051,radii_cnst, F_per_round, rho, mu, q)

                
                ############### Run simulations

                assert gauss_xstar_rBCH.shape[2] == rxn_tEnd.shape[0]

                pop_xstar_r1, traj_xstar_r1 = ssa (pop0, gauss_xstar_r1, rxn_tEnd, n_experiments, max_time_simulation, max_population, device)
                torch.save(torch.mean(pop_xstar_r1, dim=1, dtype=torch.double), f'xstar-r1_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                torch.save(torch.var(pop_xstar_r1.to(torch.float), dim=1), f'xstar-r1_popvar_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                torch.save(torch.mean(traj_xstar_r1, dim=1, dtype=torch.double), f'xstar-r1_traj_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                
                pop_xstar_rBCH, traj_xstar_rBCH = ssa (pop0, gauss_xstar_rBCH, rxn_tEnd, n_experiments, max_time_simulation, max_population, device)
                torch.save(torch.mean(pop_xstar_rBCH, dim=1, dtype=torch.double), f'xstar-rBCH_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                torch.save(torch.var(pop_xstar_rBCH.to(torch.float), dim=1), f'xstar-rBCH_popvar_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                torch.save(torch.mean(traj_xstar_rBCH, dim=1, dtype=torch.double), f'xstar-rBCH_traj_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')

                pop_x105_r1, traj_x105_r1 = ssa (pop0, gauss_x105_r1, rxn_tEnd, n_experiments, max_time_simulation, max_population, device)
                torch.save(torch.mean(pop_x105_r1, dim=1, dtype=torch.double), f'x105-r1_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                torch.save(torch.var(pop_x105_r1.to(torch.float), dim=1), f'x105-r1_popvar_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                torch.save(torch.mean(traj_x105_r1, dim=1, dtype=torch.double), f'x105-r1_traj_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')

                pop_x051_r1, traj_x051_r1 = ssa (pop0, gauss_x051_r1, rxn_tEnd, n_experiments, max_time_simulation, max_population, device)
                torch.save(torch.mean(pop_x051_r1, dim=1, dtype=torch.double), f'x051-r1_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                torch.save(torch.var(pop_x051_r1.to(torch.float), dim=1), f'x051-r1_popvar_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                torch.save(torch.mean(traj_x051_r1, dim=1, dtype=torch.double), f'x051-r1_traj_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')


In [ ]:
# for xstar-kappa=0

for Ftot_index in np.arange(F_tot_list.size):
    for pop0_center_index in np.arange(pop0_center_list.size):
        for pop0_sigma_index in np.arange (pop0_sigma_list.size): 
            
            F_tot = F_tot_list[Ftot_index]
            pop0_center = -L + pop0_center_list[pop0_center_index]    
            pop0_sigma = pop0_sigma_list[pop0_sigma_index]
            pop0 = initial_pop_gaussian(n_bins, pop0_tot, L+pop0_center, pop0_sigma)
    
            
            for n_iterations_index in np.arange (n_iterations_list.size):
                
                n_iterations = n_iterations_list[n_iterations_index]
                n_regimes = 2 * n_iterations
                print(f'Ftot={F_tot}, mu0={pop0_center}, sigma0={pop0_sigma}, rounds={n_iterations} ')

                dt_fit = max_time/n_iterations
                dt_mu = max_time/n_iterations

                F_per_round = np.ones(n_iterations)*(F_tot)
            
                rxn_tEnd = np.cumsum(np.tile([dt_fit, dt_mu], n_iterations))         
                radii_cnst =  np.ones(n_iterations)
                radii_lin =  (3*D*dt_mu) * np.arange(n_iterations, 0, -1)      
                radii_BCH = np.maximum ( np.sqrt(radii_lin), radii_cnst)
                centers_xstar_kappa0 = L + (pop0_center)*(1- (np.arange(0,n_iterations)/n_iterations))
                
                
                #############   create rxn matrices 
                
                gauss_xstar_kappa0_r1 = gauss_reaction_matrix (n_bins, n_iterations, centers_xstar_kappa0, radii_cnst, F_per_round, rho, mu, q)


                ############### Run simulations

                assert  gauss_xstar_kappa0_r1.shape[2] == rxn_tEnd.shape[0]

                pop_xstar_kappa0_r1, traj_xstar_kappa0_r1 = ssa (pop0, gauss_xstar_kappa0_r1, rxn_tEnd, n_experiments, max_time_simulation, max_population, device)
                torch.save(torch.mean(pop_xstar_kappa0_r1, dim=1, dtype=torch.double), f'xstar-kappa0-r1_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                torch.save(torch.var(pop_xstar_kappa0_r1.to(torch.float), dim=1), f'xstar-kappa0-r1_popvar_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
                torch.mean(pop_xstar_kappa0_r1, dim=1, dtype=torch.double).numpy().astype('float64').tofile(f'dat-xstar-kappa0-r1_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.dat')
                        

In [ ]:
##################  For nf0-vs-sigma0 plots

pop0_sigma_list_2 = np.array([1,2,3,4,5])       # width of the initial population (Gaussian shape)      
n_experiments_2 = 5000

for pop0_sigma_index_2 in np.arange (pop0_sigma_list_2.size): 
         
        F_tot = 0.08
        pop0_center = 10      # where the initial population is centered at; this is the x value (the index is L+8)
        
        pop0_tot = 10                           # initial total population
        n_iterations=200 
        n_regimes = 2*n_iterations
        
        pop0_sigma = pop0_sigma_list_2[pop0_sigma_index_2]
        pop0 = initial_pop_gaussian(n_bins, pop0_tot, L+pop0_center, pop0_sigma)
        
        print(f'Ftot={F_tot}, mu0={pop0_center}, sigma0={pop0_sigma}, rounds={n_iterations} ')

        dt_fit = max_time/n_iterations
        dt_mu = max_time/n_iterations

        F_per_round = np.ones(n_iterations)*(F_tot)
        rxn_tEnd = np.cumsum(np.tile([dt_fit, dt_mu], n_iterations))  
        radii_cnst =  np.ones(n_iterations)
        centers_xstar = L + (pop0_center/(1+ (pop0_sigma**2/(n_iterations*2*D*dt_mu) ))) * (1- (np.arange(0,n_iterations)/n_iterations))
        centers_xstar_kappa0 = L + (pop0_center)*(1- (np.arange(0,n_iterations)/n_iterations))
        
        #############   create rxn matrices 

        gauss_xstar_r1 = gauss_reaction_matrix (n_bins, n_iterations, centers_xstar,radii_cnst, F_per_round, rho, mu, q)
        gauss_xstar_kappa0_r1 = gauss_reaction_matrix (n_bins, n_iterations, centers_xstar_kappa0, radii_cnst, F_per_round, rho, mu, q)

        ############### Run simulations

        assert gauss_xstar_r1.shape[2] == rxn_tEnd.shape[0]

        pop_xstar_r1, traj_xstar_r1 = ssa (pop0, gauss_xstar_r1, rxn_tEnd, n_experiments_2, max_time_simulation, max_population, device)
        torch.save(torch.mean(pop_xstar_r1, dim=1, dtype=torch.double), f'xstar-r1_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
        torch.save(torch.var(pop_xstar_r1.to(torch.float), dim=1), f'xstar-r1_popvar_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
        
        torch.mean(pop_xstar_r1, dim=1, dtype=torch.double).numpy().astype('float64').tofile(f'dat-xstar-r1_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.dat')

        pop_xstar_kappa0_r1, traj_xstar_kappa0_r1 = ssa (pop0, gauss_xstar_kappa0_r1, rxn_tEnd, n_experiments_2, max_time_simulation, max_population, device)
        torch.save(torch.mean(pop_xstar_kappa0_r1, dim=1, dtype=torch.double), f'xstar-kappa0-r1_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
        torch.save(torch.var(pop_xstar_kappa0_r1.to(torch.float), dim=1), f'xstar-kappa0-r1_popvar_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
        
        torch.mean(pop_xstar_kappa0_r1, dim=1, dtype=torch.double).numpy().astype('float64').tofile(f'dat-xstar-kappa0-r1_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.dat')


In [ ]:
##################  For nf0-vs-sigmaMin plots

sigmamin_list = np.array([1,2,3,4,5])

for sigmamin_index in np.arange (sigmamin_list.size): 
        sigmamin = sigmamin_list[sigmamin_index]
        F_tot = 0.08
        pop0_center = 8       # where the initial population is centered at; this is the x value (the index is L+8)
        pop0_tot = 10                           # initial total population
        n_iterations=200 
        n_regimes = 2 * n_iterations
        
        pop0_sigma = 1
        pop0 = initial_pop_gaussian(n_bins, pop0_tot, L+pop0_center, pop0_sigma)
        
        print(f'Ftot={F_tot}, mu0={pop0_center}, sigma0={pop0_sigma}, rounds={n_iterations} ')

        dt_fit = max_time/n_iterations
        dt_mu = max_time/n_iterations

        F_per_round = np.ones(n_iterations)*(F_tot)
        # other choice: # F_per_round = np.ones(n_iterations)*(F_tot/n_iterations)
        # Not dividing makes more sense since the overall total given fitness is this value integrated over time and over the bins & dividing the time into many smaller pieces would not change the total F then

        rxn_tEnd = np.cumsum(np.tile([dt_fit, dt_mu], n_iterations))          # these are only for the sake of simulation

        radii_cnst =  sigmamin * np.ones(n_iterations)
        radii_lin =  (3*D*dt_mu) * np.arange(n_iterations, 0, -1)      # starts large and goes small
        radii_BCH = np.maximum ( np.sqrt(radii_lin), radii_cnst)
        centers_xstar = L + (pop0_center/(1+ ( pop0_sigma**2/(n_iterations*2*D*dt_mu) ))) * (1- (np.arange(1,n_iterations+1)/n_iterations))
        centers_xstar_kappa0 = L + (pop0_center)*(1- (np.arange(1,n_iterations+1)/n_iterations))
        
        #############   create rxn matrices 

        gauss_xstar_rmin = gauss_reaction_matrix (n_bins, n_iterations, centers_xstar,radii_cnst, F_per_round, rho, mu, q)
        gauss_xstar_rBCH = gauss_reaction_matrix (n_bins, n_iterations, centers_xstar, radii_BCH, F_per_round, rho, mu, q)


        ############### Run simulations

        assert gauss_xstar_rmin.shape[2] == rxn_tEnd.shape[0]
        # number of regimes should be the same (and is twice the number of rounds)


        ###########################################

        pop_xstar_rmin, traj_xstar_rmin = ssa (pop0, gauss_xstar_rmin, rxn_tEnd, n_experiments, max_time_simulation, max_population, device)
        torch.save(torch.mean(pop_xstar_rmin, dim=1, dtype=torch.double), f'xstar-rmin={sigmamin}_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
        torch.save(torch.var(pop_xstar_r1.to(torch.float), dim=1), f'xstar-rmin={sigmamin}_popvar_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
        
        torch.mean(pop_xstar_rmin, dim=1, dtype=torch.double).numpy().astype('float64').tofile(f'dat-xstar-rmin={sigmamin}_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.dat')

        pop_xstar_rBCH, traj_xstar_rBCH = ssa (pop0, gauss_xstar_rBCH, rxn_tEnd, n_experiments, max_time_simulation, max_population, device)
        torch.save(torch.mean(pop_xstar_rBCH, dim=1, dtype=torch.double), f'xstar-rBCH_rmin={sigmamin}_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
        torch.save(torch.var(pop_xstar_rBCH.to(torch.float), dim=1), f'xstar-rBCH_rmin={sigmamin}_popvar_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.pt')
        
        torch.mean(pop_xstar_rBCH, dim=1, dtype=torch.double).numpy().astype('float64').tofile(f'dat-xstar-rBCH_rmin={sigmamin}_pop_Ftot={F_tot}_mu0={pop0_center}_sigma0={pop0_sigma}_rounds={n_iterations}.dat')